In [1]:
# Import libraries
import numpy as np
import pandas as pd
from keras.layers import Dropout
import os
import cv2
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import keras
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
%matplotlib inline

Using TensorFlow backend.


In [2]:
# load files
train = pd.read_csv(r'C:\Users\sdinu\OneDrive\Documents\02 MS BA\04 Fall 2017\IDS594 Machine Learning with Python\02 Project\01 Dataset\train.csv')
test = pd.read_csv(r'C:\Users\sdinu\OneDrive\Documents\02 MS BA\04 Fall 2017\IDS594 Machine Learning with Python\02 Project\01 Dataset\test.csv')

In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [7]:
# set path to read train and test image
TRAIN_PATH = 'C:/Users/sdinu/OneDrive/Documents/02 MS BA/04 Fall 2017/IDS594 Machine Learning with Python/02 Project/01 Dataset/train_img/'
TEST_PATH = 'C:/Users/sdinu/OneDrive/Documents/02 MS BA/04 Fall 2017/IDS594 Machine Learning with Python/02 Project/01 Dataset/test_img/'

In [8]:
# function to read images as arrays
def read_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128,128)) # you can resize to  (128,128) or (256,256)
    return img

In [9]:
# Read images
train_data = []
test_data = []
train_labels = train['label'].values

for img in tqdm(train['image_id'].values):
    train_data.append(read_image(TRAIN_PATH + '{}.png'.format(img)))

train_array = np.asarray(train_data)
    
for img in tqdm(test['image_id'].values):
    test_data.append(read_image(TEST_PATH + '{}.png'.format(img)))

test_array = np.asarray(test_data)
    
train_array = train_array.reshape(train_array.shape[0], 128, 128, 3).astype('float32')
test_array = test_array.reshape(test_array.shape[0], 128, 128, 3).astype('float32')

train_array /= 255
test_array /= 255
print(train_array.shape)
print(test_array.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 965/965 [00:10<00:00, 94.81it/s]


(2250, 128, 128, 3)
(965, 128, 128, 3)


In [10]:
# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]

label_list = test['label'].tolist()
Y_test = {k:v for v,k in enumerate(set(label_list))}
y_test = [Y_test[k] for k in label_list]

In [11]:
num_classes = 25
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [20]:
# Parameter tuning - Optimizer selection

# Defining model
def CNN_optimizer(optimizer='SGD'):
	model = Sequential()
	model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu', kernel_initializer='uniform',
                 input_shape=(128, 128, 3)))
	model.add(MaxPooling2D(pool_size=(2, 2)))
    
	model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='uniform'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Flatten())
	model.add(Dense(64, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))

	model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
	return model

In [21]:
# create model
model = KerasClassifier(CNN_optimizer, epochs=10, verbose=0)

In [22]:
# Define the grid search parameters - Optimization Algorithms

from sklearn.model_selection import GridSearchCV
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(model, param_grid=param_grid, n_jobs=1, 
                    scoring='neg_log_loss')
grid_result = grid.fit(train_array, y_train)

In [23]:
# Summarize results - Optimization Algorithms
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -2.932136 using {'optimizer': 'Adagrad'}
-3.108464 (0.024585) with: {'optimizer': 'SGD'}
-4.161799 (0.200477) with: {'optimizer': 'RMSprop'}
-2.932136 (0.071840) with: {'optimizer': 'Adagrad'}
-3.164707 (0.322411) with: {'optimizer': 'Adadelta'}
-3.975320 (0.175443) with: {'optimizer': 'Adam'}
-3.093017 (0.203063) with: {'optimizer': 'Adamax'}
-6.275841 (0.733833) with: {'optimizer': 'Nadam'}


In [24]:
# Parameter tuning - Epoch and Batchsize

# Defining model
def CNN_epoch_batchsize():
	model = Sequential()
	model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu', kernel_initializer='uniform',
                 input_shape=(128, 128, 3)))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='uniform'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
    
	model.add(Flatten())
	model.add(Dense(64, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))

	model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='Adagrad',
              metrics=['accuracy'])
	return model

In [25]:
# create model
model = KerasClassifier(CNN_epoch_batchsize, verbose=0)

In [26]:
# Define the grid search parameters - Epochs and Batch Size

batch_size = [64, 128 , 256]
epochs = [5, 10, 15]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, 
                    scoring='neg_log_loss')
grid_result = grid.fit(train_array, y_train)

In [27]:
# Summarize results - Epochs and Batch Size

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -2.822614 using {'batch_size': 256, 'epochs': 15}
-12.178999 (12.875563) with: {'batch_size': 64, 'epochs': 5}
-12.648912 (13.498463) with: {'batch_size': 64, 'epochs': 10}
-12.535687 (13.092475) with: {'batch_size': 64, 'epochs': 15}
-3.181633 (0.119573) with: {'batch_size': 128, 'epochs': 5}
-2.862587 (0.018580) with: {'batch_size': 128, 'epochs': 10}
-3.037257 (0.060851) with: {'batch_size': 128, 'epochs': 15}
-3.224295 (0.094919) with: {'batch_size': 256, 'epochs': 5}
-3.125799 (0.067949) with: {'batch_size': 256, 'epochs': 10}
-2.822614 (0.087138) with: {'batch_size': 256, 'epochs': 15}


In [12]:
from keras.optimizers import Adagrad

In [35]:
# Parameter tuning - Learning Rate

def CNN_learningRate(learn_rate=0.1):
	model = Sequential()
	model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu', kernel_initializer='uniform',
                 input_shape=(128, 128, 3)))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='uniform'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
    
	model.add(Flatten())
	model.add(Dense(64, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
    
	optimizer = Adagrad(lr=learn_rate)
	model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
	return model

In [36]:
# create model
model = KerasClassifier(CNN_learningRate, epochs=15, batch_size=256, verbose=0)

In [37]:
# Define the grid search parameters - Learning Rate
learn_rate = [0.1, 0.075, 0.005]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, 
                    scoring='neg_log_loss')
grid_result = grid.fit(train_array, y_train)

In [38]:
# Summarize results - Learning Rate
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -2.907828 using {'learn_rate': 0.005}
-31.944531 (0.207090) with: {'learn_rate': 0.1}
-32.236191 (0.075202) with: {'learn_rate': 0.075}
-2.907828 (0.099609) with: {'learn_rate': 0.005}


In [39]:
# Parameter tuning - Kernel Initializer

def CNN_kernelinit(init_mode='uniform'):
	model = Sequential()
	model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu', kernel_initializer=init_mode,
                 input_shape=(128, 128, 3)))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer=init_mode))
	model.add(MaxPooling2D(pool_size=(2, 2)))
    
	model.add(Flatten())
	model.add(Dense(64, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
    
	optimizer = Adagrad(lr=0.005)
	model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
	return model

In [40]:
# create model
model = KerasClassifier(CNN_kernelinit, epochs=15, batch_size=256, verbose=0)

In [41]:
# Define the grid search parameters - Kernel Initializer
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 
             'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, 
                    scoring='neg_log_loss')
grid_result = grid.fit(train_array, y_train)

In [42]:
# Summarize results - Kernel Initializer
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -3.184707 using {'init_mode': 'zero'}
-32.159438 (0.287183) with: {'init_mode': 'uniform'}
-33.019070 (0.533087) with: {'init_mode': 'lecun_uniform'}
-32.190140 (0.321264) with: {'init_mode': 'normal'}
-3.184707 (0.009047) with: {'init_mode': 'zero'}
-32.712059 (0.735226) with: {'init_mode': 'glorot_normal'}
-32.282243 (0.362611) with: {'init_mode': 'glorot_uniform'}
-32.098036 (0.228718) with: {'init_mode': 'he_normal'}
-32.543203 (0.402056) with: {'init_mode': 'he_uniform'}
